# Kiểm tra GPU

In [1]:
!nvidia-smi

Tue May  7 15:13:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Cài đặt và import các thư viện cần thiết

In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Restart runtime rồi chạy code phía dưới

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np

# Load model và tokenizer

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("FPTAI/vibert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/581M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

# Dataset

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd

df1_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_victsd.csv')
df1_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_victsd.csv')
df1_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_victsd.csv')

df2_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_vihsd.csv')
df2_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_vihsd.csv')
df2_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_vihsd.csv')

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df1 = pd.concat([df1a, df1b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df1.reset_index(drop=True, inplace=True)

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df2 = pd.concat([df2a, df2b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df2.reset_index(drop=True, inplace=True)

In [8]:
df2_train.head()

,text,label
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [9]:
def process_df2(df):
    df['label'] = df['label'].replace(2, 1)
    return df.dropna()

df2_train = process_df2(df2_train)
df2_valid = process_df2(df2_valid)
df2_test = process_df2(df2_test)

In [10]:
df2_train.isna().sum()

text     0
label    0
dtype: int64

In [11]:
def process_df1(df):
    return df.dropna()
df1_train = process_df1(df1_train)
df1_valid = process_df1(df1_valid)
df1_test = process_df1(df1_test)

In [12]:
df1_valid.isna().sum()

text     0
label    0
dtype: int64

In [13]:
tu_dien_viet_tat = {
    "a": "anh",
    "c": "chị",
    "e": "em",
    "b": "bạn",
    "bn": "bạn",
    "t": "tao",
    "m": "mày",
    "cmt": "bình luận",
    "ng": "người",
    "mn": "mọi người",
    "iu": "yêu",
    "ct": "chia tay",
    "ny": "người yêu",
    "vk": "vợ",
    "ck": "chồng",
    "vc": "vợ chồng",
    "ox": "ông xã",
    "bx": "bà xã",
    "h": "giờ",
    "g": "giờ",
    "ph": "phút",
    "bh": "bây giờ",
    "bjo": "bây giờ",
    "hn": "hôm nay",
    "hnay": "hôm nay",
    "hqua": "hôm qua",
    "htrc": "hôm trước",
    "nt": "nhắn tin",
    "nc": "nói chuyện",
    "mún": "muốn",
    "bit": "biết",
    "bik": "biết",
    "cb": "chuẩn bị",
    "nch": "nói chung",
    "dt": "điện thoại",
    "đt": "điện thoại",
    "bth": "bình thường",
    "j": "gì",
    "bn": "bao nhiêu",
    "ntn": "như thế nào",
    "k": "không",
    "ko": "không",
    "hok": "không",
    "hem": "không",
    "dc": "được",
    "đc": "được",
    "r": "rồi",
    "rùi": "rồi",
    "nhìu": "nhiều",
    "wá": "quá",
    "cg": "cũng",
    "uk": "ừ",
    "uh": "ừ",
    "ukm": "ừ",
    "G9": "ngủ ngon",
    "hic": "tiếng khóc",
    "vn": "Việt Nam"
}

In [14]:
!pip install emoji_vietnamese

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 2.7 MB/s eta 0:00:00


In [15]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 61.0 MB/s eta 0:00:00


In [16]:
import underthesea
import string
import emoji_vietnamese

def preprocess_data(data):
    data = data.replace(
        r'http\S+', '').replace(r'www\S+', '')
    data = data.replace(
        '['+string.punctuation+']', '')
    data = data.lower().strip()
    data = emoji_vietnamese.demojize(data)
    new_str = data.split()
    for i in range(len(new_str)):
       if new_str[i] in tu_dien_viet_tat.keys():
         new_str[i] = tu_dien_viet_tat[new_str[i]]
    return underthesea.word_tokenize(" ".join(new_str), format='text')


In [17]:
df1_train['text'] = df1_train['text'].map(preprocess_data)
df1_valid['text'] = df1_valid['text'].map(preprocess_data)
df1_test['text'] = df1_test['text'].map(preprocess_data)

df2_train['text'] = df2_train['text'].map(preprocess_data)
df2_valid['text'] = df2_valid['text'].map(preprocess_data)
df2_test['text'] = df2_test['text'].map(preprocess_data)

In [18]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42, sampling_strategy=1)
X_resampled, y_resampled = ros.fit_resample(df1_train[['text']], df1_train['label'])
df1_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df1_train_new.columns = ['text', 'label']

X_resampled, y_resampled = ros.fit_resample(df2_train[['text']], df2_train['label'])
df2_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df2_train_new.columns = ['text', 'label']

In [19]:
from datasets import Dataset

def convert_to_datasets(df):
    dataset = Dataset.from_pandas(df).shuffle(seed=20)
    return dataset

train_dataset1 = convert_to_datasets(df1_train_new)
valid_dataset1 = convert_to_datasets(df1_valid)
test_dataset1 = convert_to_datasets(df1_test)

train_dataset2 = convert_to_datasets(df2_train_new)
valid_dataset2 = convert_to_datasets(df2_valid)
test_dataset2 = convert_to_datasets(df2_test)

In [20]:
train_dataset1

Dataset({
    features: ['text', 'label'],
    num_rows: 12482
})

In [21]:
# Tokenizer dataset và thêm padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=256)
tokenized_train_dataset1 = train_dataset1.map(tokenize_function, batched=True, remove_columns='text')
tokenized_valid_dataset1 = valid_dataset1.map(tokenize_function, batched=True, remove_columns='text')
tokenized_test_dataset1 = test_dataset1.map(tokenize_function, batched=True, remove_columns='text')

tokenized_train_dataset2 = train_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_valid_dataset2 = valid_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_test_dataset2 = test_dataset2.map(tokenize_function, batched=True, remove_columns='text')
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/12482 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/39770 [00:00<?, ? examples/s]

Map:   0%|          | 0/2672 [00:00<?, ? examples/s]

Map:   0%|          | 0/6680 [00:00<?, ? examples/s]

# Training

In [22]:
# Viết hàm tính accuracy
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-22-b113ca1a6718>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset1,
    eval_dataset=tokenized_valid_dataset1,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)



In [24]:
trainer_1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.360500,0.406511,0.871500
2,0.090200,0.714943,0.886000
3,0.021500,0.794023,0.888500


TrainOutput(global_step=2343, training_loss=0.15741163148770107, metrics={'train_runtime': 1670.539, 'train_samples_per_second': 22.416, 'train_steps_per_second': 1.403, 'total_flos': 4926228289505280.0, 'train_loss': 0.15741163148770107, 'epoch': 3.0})

In [25]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics_f1(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # precision_recall_fscore_support returns a tuple containing precision, recall, f1, support respectively
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset1,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()


{'eval_loss': 0.424488365650177,
 'eval_precision': 0.42735042735042733,
 'eval_recall': 0.45454545454545453,
 'eval_f1': 0.44052863436123346,
 'eval_runtime': 15.3898,
 'eval_samples_per_second': 64.978,
 'eval_steps_per_second': 8.122}

In [26]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset2,
    eval_dataset=tokenized_valid_dataset2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

trainer_2.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359500,0.398955,0.882859
2,0.165700,0.559776,0.883234


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359500,0.398955,0.882859
2,0.165700,0.559776,0.883234
3,0.084100,0.677553,0.879865


TrainOutput(global_step=7458, training_loss=0.20310397568833383, metrics={'train_runtime': 5064.4802, 'train_samples_per_second': 23.558, 'train_steps_per_second': 1.473, 'total_flos': 1.56958900075008e+16, 'train_loss': 0.20310397568833383, 'epoch': 3.0})

In [28]:
trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset2,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()

{'eval_loss': 0.3940124809741974,
 'eval_precision': 0.6477987421383647,
 'eval_recall': 0.6369257950530035,
 'eval_f1': 0.6423162583518931,
 'eval_runtime': 104.0956,
 'eval_samples_per_second': 64.172,
 'eval_steps_per_second': 8.021}

# Predict

In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
pipe("Quang bị ngu à ")

In [30]:
model.num_parameters()

115355906

In [34]:
#load model
from transformers import AutoModel

# Đường dẫn tới thư mục chứa mô hình
model_path = "./results1"

# Tải mô hình
model = AutoModelForSequenceClassification.from_pretrained(model_path)

OSError: ./results1 does not appear to have a file named config.json. Checkout 'https://huggingface.co/./results1/tree/None' for available files.